In [3]:
from kafka import KafkaConsumer

# Define server with port
bootstrap_servers = ['localhost:29092']

# Define topic name from where the message will recieve
topicName = 'source.public.raw_grades'

# Initialize consumer variable
consumer = KafkaConsumer (topicName , auto_offset_reset='earliest', bootstrap_servers = bootstrap_servers)

# Read and print message from consumer
for msg in consumer:
    print("Topic Name=%s,Message=%s"%(msg.topic,msg.value))

KeyboardInterrupt: 

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark Session with Hudi configuration
spark = (SparkSession.builder
         .appName("HudiTableConsumer")
         .master("local[*]")
         .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
         .config('spark.jars.packages', 'org.apache.hudi:hudi-spark3.3-bundle_2.12:0.14.0')
         .config("spark.sql.extensions", "org.apache.hudi.spark3.sql.HoodieSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog")
         .config("spark.driver.host", "localhost")  # Add this line
         .config("spark.driver.bindAddress", "localhost")  # Add this line
         .getOrCreate())

# Read from Hudi table
hudi_df = (spark.read
           .format("hudi")
           .load("C:/tmp/spark_warehouse/from_kafka"))

# Show schema and sample data
print("Table Schema:")
hudi_df.printSchema()

print("\nSample Data:")
hudi_df.show(5)

# Get total count
print(f"\nTotal Records: {hudi_df.count()}")

# Optional: Get statistics by schoolyear (partition column)
print("\nRecords by Schoolyear:")
hudi_df.groupBy("schoolyear").count().show()

Table Schema:
root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- semester: string (nullable = true)
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- units: integer (nullable = true)
 |-- instructor_id: string (nullable = true)
 |-- instructor_name: string (nullable = true)
 |-- srcode: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- campus: string (nullable = true)
 |-- college: string (nullable = true)
 |-- program: string (nullable = true)
 |-- major: string (nullable = true)
 |-- yearlevel: string (nullable = true)
 |-- curriculum: string (nullable = true)
 |-- class_section: string (nullable = true)
 |-- grade_final: string (nullable = true)
 |-- grade_reexam: string (nullable = t